In [1]:
import numpy as np
import pandas as pd

import os

from tqdm import tqdm

In [2]:
raw_data = pd.read_csv('/opt/ml/input/data/FE_total_data.csv')
raw_data.sort_values(by=["userID", "Timestamp"], inplace=True)
df = raw_data.copy()

In [3]:
df["grade"] = df["testId"].apply(lambda x: int(x[1:4]) // 10)

o_df = df[df["answerCode"] == 1]
x_df = df[df["answerCode"] == 0]

In [4]:
df["Grade_o"] = (
    df.groupby(["userID", "grade"])["answerCode"]
    .transform(lambda x: x.cumsum().shift(1))
    .fillna(0)
)

df["GradeCount"] = df.groupby(["userID", "grade"]).cumcount()
df["GradeAcc"] = (df["Grade_o"] / df["GradeCount"]).fillna(0)

In [5]:
df.to_csv('/opt/ml/input/data/FE/GradeAcc.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "GradeAcc"], index=False)
df.to_csv('/opt/ml/input/data/FE/Grade_o.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "Grade_o"], index=False)
df.to_csv('/opt/ml/input/data/FE/GradeCount.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "GradeCount"], index=False)